In [ ]:
#! pip install -r requirement.txt

In [35]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv
import time
import os
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

cohere_api_key = os.getenv("COHERE_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

In [36]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.llms import cohere
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Cohere
from langchain_cohere import ChatCohere
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
import chromadb


In [2]:
#pip install langchain_huggingface
#!pip install langchain-chroma

In [38]:
loader=PyPDFLoader("C:\\Users\\SAUNAK MITRA\\Downloads\\NIPS-2017-attention-is-all-you-need-Paper.pdf")
document=loader.load()

In [39]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
texts=text_splitter.split_documents(document)

In [40]:
len(texts)

453

In [23]:
persist_directory='db'
embedding=HuggingFaceEmbeddings()
vectordb=Chroma.from_documents(documents=texts,embedding=embedding,persist_directory=persist_directory)


In [24]:
retriever=vectordb.as_retriever()

In [8]:
docs=retriever.invoke("what is the data about?")

In [9]:
len(docs)

4

In [10]:
retriever.search_type

'similarity'

In [11]:
#pip install langchain-cohere

In [12]:
qa_chain=RetrievalQA.from_chain_type(llm=Cohere(cohere_api_key=cohere_api_key),retriever=retriever,return_source_documents=True)

C:\Users\SAUNAK MITRA\AppData\Local\Temp\ipykernel_19168\1234360599.py:1: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  qa_chain=RetrievalQA.from_chain_type(llm=Cohere(cohere_api_key ="VzogaFZxHAsgDHVZPn8eTRXN0aOJACvbkeNjzvaw"),retriever=retriever,return_source_documents=True)


In [13]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [14]:
query="what is the heading of this paper?"
llm_response=qa_chain(query)
process_llm_response(llm_response)

C:\Users\SAUNAK MITRA\AppData\Local\Temp\ipykernel_19168\286441543.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_response=qa_chain(query)


 The heading of this paper is "Comments, Corrections and Inspiration" and it is repeated twice. 
However, I do not have access to any additional context beyond what has been provided regarding the matter being discussed in the article or the document's main focus. 

Can I assist you with anything else with the information that I have been given? 


Sources:
C:\Users\SAUNAK MITRA\Downloads\NIPS-2017-attention-is-all-you-need-Paper.pdf
C:\Users\SAUNAK MITRA\Downloads\NIPS-2017-attention-is-all-you-need-Paper.pdf
C:\Users\SAUNAK MITRA\Downloads\NIPS-2017-attention-is-all-you-need-Paper.pdf
C:\Users\SAUNAK MITRA\Downloads\NIPS-2017-attention-is-all-you-need-Paper.pdf


In [25]:
from langchain.prompts import ChatPromptTemplate

In [26]:
template="""You are an assistant for question_answering tasks.
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer ,just say you don't know.
Use ten sentences maximum and keep the answer concise. 
Question:{question}
context: {context}
Answer:
"""

In [27]:
prompt=ChatPromptTemplate.from_template(template)

In [28]:
from langchain.schema.runnable import  RunnablePassthrough
from langchain.schema.output_parser import  StrOutputParser

In [29]:
llm=Cohere(cohere_api_key=cohere_api_key)

In [30]:
rag_chain = (
                        {"context": retriever , "question": RunnablePassthrough()}
                        | prompt
                        | llm
                        | StrOutputParser()
)

In [34]:
rag_chain.invoke("what is the paper about?")

' I don\'t know the answer, but I can extract information from the provided context that might be relevant to the question. \n\nThe context seems to refer to a paper titled "Attention is All You Need". The paper\'s pages are mentioned as 770–778, 2016. It seems to be related to machine learning and recognition. \n\nI hope this information is helpful, please provide additional context or questions that I can use to further refine my search. '

In [32]:
llm1= ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
rag_chain1 = (
                        {"context": retriever , "question": RunnablePassthrough()}
                        | prompt
                        | llm1
                        | StrOutputParser()
)

In [33]:
rag_chain1.invoke("what is the heading of this paper?")

'Based on the provided context, I cannot determine the exact heading of the paper as it is not included in the given sentences. The context mentions that comments, corrections, and inspiration can be found on page 8, and section 3.2 describes something, but it does not specify what.'

In [34]:
llm_response

{'query': 'what is the heading of this paper?',
 'result': ' The heading of the paper is "EffectiveRecognition of Vedic Languages". \n\nIt is likely 3.2 refers to section 3.2, "Post Processing Module" in the document. ',
 'source_documents': [Document(metadata={'page': 8, 'source': 'C:\\Users\\SAUNAK MITRA\\Downloads\\NIPS-2017-attention-is-all-you-need-Paper.pdf'}, page_content='comments, corrections and inspiration.\n9'),
  Document(metadata={'page': 1, 'source': 'C:\\Users\\SAUNAK MITRA\\Downloads\\NIPS-2017-attention-is-all-you-need-Paper.pdf'}, page_content='described in section 3.2.'),
  Document(metadata={'page': 9, 'source': 'C:\\Users\\SAUNAK MITRA\\Downloads\\NIPS-2017-attention-is-all-you-need-Paper.pdf'}, page_content='Recognition, pages 770–778, 2016.'),
  Document(metadata={'page': 9, 'source': 'C:\\Users\\SAUNAK MITRA\\Downloads\\NIPS-2017-attention-is-all-you-need-Paper.pdf'}, page_content='machine translation. CoRR, abs/1406.1078, 2014.')]}

In [24]:
qa_chain1=RetrievalQA.from_chain_type(llm=ChatGroq(groq_api_key=groq_api_key),retriever=retriever,return_source_documents=True)

In [35]:
query="what is the heading of this paper?"
llm_response=qa_chain1(query)
process_llm_response(llm_response)

The heading of the paper "machine translation. CoRR, abs/1406.1078, 2014" is "machine translation". The other pieces of context don't provide any additional information that would help answer the question.


Sources:
C:\Users\SAUNAK MITRA\Downloads\NIPS-2017-attention-is-all-you-need-Paper.pdf
C:\Users\SAUNAK MITRA\Downloads\NIPS-2017-attention-is-all-you-need-Paper.pdf
C:\Users\SAUNAK MITRA\Downloads\NIPS-2017-attention-is-all-you-need-Paper.pdf
C:\Users\SAUNAK MITRA\Downloads\NIPS-2017-attention-is-all-you-need-Paper.pdf
